# Economics-specific packages

Based on a notebook by Chase Coleman and Spencer Lyon and on material from QuantEcon

18 December 2017

This lecture will focus mostly on QuantEcon. Other economics packages of interest include DSGE.jl and StateSpaceRoutines.jl, which will be covered separately.

## QuantEcon.jl

What is QuantEcon?

- Set of [lectures](https://lectures.quantecon.org/) by Tom Sargent and John Stachurski to teach computational economics and programming principles
- A [community](http://discourse.quantecon.org) aimed at teaching best practices and encouraging collaboration
- A set of software libraries in [Python](https://quantecon.org/quantecon-py) and [Julia](https://quantecon.org/quantecon-jl) that implement common numerical routines used in economic research
- [QuantEcon.jl](https://github.com/QuantEcon/QuantEcon.jl) is the Julia version of the software library

In [1]:
# Pkg.add("QuantEcon")

### QuantEcon.jl Basics

In [1]:
using QuantEcon

#### Markov Chains

QuantEcon.jl has rich support for working with discrete state MarkovChains

In [2]:
mc = MarkovChain([0.9 0.1; 0.2 0.8])

Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
[0.9 0.1; 0.2 0.8]

In [4]:
stationary_distributions(mc)

1-element Array{Array{Float64,1},1}:
 [0.666667, 0.333333]

In [3]:
?simulate

search: simulate simulate! simulate_indices simulate_indices! simulation



Simulate one sample path of the Markov chain `mc`. The resulting vector has the state values of `mc` as elements.

### Arguments

  * `mc::MarkovChain` : MarkovChain instance.
  * `ts_length::Int` : Length of simulation
  * `;init::Int=rand(1:n_states(mc))` : Initial state

### Returns

  * `X::Vector` : Vector containing the sample path, with length ts_length


In [4]:
mean(simulate(mc, 10000))  # should be roughly 2/3*1 + 1/3*2 = 4/3

1.3358

In [6]:
# can also pass state_values as second argument
mc2 = MarkovChain(
    mc.p,  # re-use transition matrix
    [0.0, 42.0]
)

Discrete Markov Chain
stochastic matrix of type Array{Float64,2}:
[0.9 0.1; 0.2 0.8]

In [8]:
mean(simulate(mc2, 100000)) # should be roughly 2/3*0 + 1/3*42 = 14

13.91544

In [7]:
# can also simulate the indices
simulate_indices(mc2, 10, init=2)

10-element Array{Int64,1}:
 2
 2
 2
 2
 2
 2
 1
 1
 1
 1

In [8]:
# fill pre-allocated matrix with samples
# each column is a time-series
out = zeros(500, 10)
simulate!(out, mc2, init=2)
out

500×10 Array{Float64,2}:
 42.0  42.0  42.0  42.0  42.0  42.0  42.0  42.0  42.0  42.0
 42.0  42.0  42.0  42.0  42.0  42.0  42.0  42.0  42.0   0.0
 42.0  42.0  42.0  42.0   0.0  42.0  42.0   0.0  42.0   0.0
 42.0  42.0  42.0   0.0   0.0   0.0  42.0   0.0  42.0   0.0
 42.0   0.0  42.0   0.0   0.0   0.0  42.0   0.0   0.0   0.0
 42.0   0.0  42.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  42.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  42.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0  42.0   0.0   0.0   0.0   0.0  42.0   0.0   0.0   0.0
  0.0  42.0   0.0   0.0   0.0   0.0  42.0   0.0   0.0   0.0
  ⋮                             ⋮                          
  0.0  42.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  42.0   0.0   0.0   0.0   0.0  42.0   0.0   0.0
  0.0   0.0  42

In [11]:
# same with indices
out_inds = zeros(Int, 500, 10)
simulate_indices!(out_inds, mc2, init=2)
out_inds

500×10 Array{Int64,2}:
 2  2  2  2  2  2  2  2  2  2
 2  1  1  2  2  1  2  1  2  2
 2  2  2  2  2  2  1  1  2  2
 1  2  2  2  2  2  1  1  2  2
 1  2  2  2  2  2  1  1  2  2
 2  2  2  2  2  2  1  1  2  2
 2  1  2  2  2  2  1  1  2  2
 1  1  2  2  2  2  1  1  2  2
 1  1  2  1  2  1  1  1  1  2
 1  1  2  1  1  1  1  2  1  1
 1  1  2  2  1  1  1  2  2  1
 1  1  2  1  1  1  1  2  2  1
 1  1  1  1  1  1  1  1  2  1
 ⋮              ⋮            
 1  1  2  1  1  2  1  2  1  2
 1  1  2  1  2  2  1  2  2  2
 1  1  2  1  2  1  2  2  2  2
 2  1  2  1  1  1  2  2  2  1
 2  2  1  2  1  1  2  1  2  1
 2  2  1  2  1  1  1  1  2  1
 1  2  1  2  1  1  1  1  2  1
 2  2  1  2  1  1  1  1  2  1
 2  1  1  2  1  2  2  1  2  1
 2  1  1  2  1  2  2  1  2  2
 2  1  1  1  1  2  2  1  2  2
 2  1  1  1  1  2  1  1  2  2

In [9]:
# other things we can do with MarkovChains
methodswith(MarkovChain)

10-element Array{Method,1}:
 period(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:220                                                                              
 communication_classes(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:177                                                               
 is_aperiodic(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:205                                                                        
 is_irreducible(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:191                                                                      
 n_states(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:61                                                                             
 recurrent_classes(mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:162                                                                   
 simulate(mc::QuantEcon.MarkovChain, ts_length::Int64) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:360                                                          
 simulate!(X::Union{AbstractArray{T,1} where T, AbstractArray{T,2} where T}, mc::QuantEcon.MarkovChain) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:385         
 simulate_indices(mc::QuantEcon.MarkovChain, ts_length::Int64) in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:415                                                  
 simulate_indices!(X::Union{AbstractArray{T,1}, AbstractArray{T,2}}, mc::QuantEcon.MarkovChain) where T<:Integer in QuantEcon at /Users/rceaxg24/.julia/v0.6/QuantEcon/src/markov/mc_tools.jl:438

#### Discretizing AR(1) process

QuantEcon.jl provides two methods for discretizing an AR(1) process into an instance of MarkovChain:


In [10]:
?tauchen

search: tauchen



Tauchen's (1996) method for approximating AR(1) process with finite markov chain

The process follows

$$
    y_t = \mu + \rho y_{t-1} + \epsilon_t
$$

where $\epsilon_t \sim N (0, \sigma^2)$

##### Arguments

  * `N::Integer`: Number of points in markov process
  * `ρ::Real` : Persistence parameter in AR(1) process
  * `σ::Real` : Standard deviation of random component of AR(1) process
  * `μ::Real(0.0)` : Mean of AR(1) process
  * `n_std::Integer(3)` : The number of standard deviations to each side the process should span

##### Returns

  * `mc::MarkovChain{Float64}` : Markov chain holding the state values and transition matrix


In [11]:
?rouwenhorst

search: rouwenhorst



Rouwenhorst's method to approximate AR(1) processes.

The process follows

$$
    y_t = \mu + \rho y_{t-1} + \epsilon_t
$$

where $\epsilon_t \sim N (0, \sigma^2)$

##### Arguments

  * `N::Integer` : Number of points in markov process
  * `ρ::Real` : Persistence parameter in AR(1) process
  * `σ::Real` : Standard deviation of random component of AR(1) process
  * `μ::Real(0.0)` :  Mean of AR(1) process

##### Returns

  * `mc::MarkovChain{Float64}` : Markov chain holding the state values and transition matrix


In [12]:
mc3 = rouwenhorst(10, 0.95, 1.0, 0.3)
@show stationary_distributions(mc3)
@show mc3.state_values
simulate(mc3, 20)

stationary_distributions(mc3) = Array{Float64,1}[[0.00195312, 0.0175781, 0.0703125, 0.164063, 0.246094, 0.246094, 0.164062, 0.0703125, 0.0175781, 0.00195312]]
mc3.state_values = -3.6076892283052313:2.1350420507344947:15.60768922830522


20-element Array{Float64,1}:
 11.3376 
 11.3376 
 11.3376 
 11.3376 
 11.3376 
 11.3376 
 11.3376 
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  9.20256
  7.06752
  4.93248
  4.93248

#### Quadrature routines

QuantEcon.jl has the family of Quadrature routines written by Miranda and Fackler in CompEcon as well as others used by Maliar, Maliar, and Judd

In [16]:
filter(x -> startswith(string(x), "qnw"), names(QuantEcon))

12-element Array{Symbol,1}:
 :qnwbeta     
 :qnwcheb     
 :qnwequi     
 :qnwgamma    
 :qnwlege     
 :qnwlogn     
 :qnwmonomial1
 :qnwmonomial2
 :qnwnorm     
 :qnwsimp     
 :qnwtrap     
 :qnwunif     

In [13]:
?qnwnorm

search: qnwnorm qnwmonomial2 qnwmonomial1



Computes nodes and weights for multivariate normal distribution.

##### Arguments

  * `n::Union{Int, Vector{Int}}` : Number of desired nodes along each dimension
  * `mu::Union{Real, Vector{Real}}` : Mean along each dimension
  * `sig2::Union{Real, Vector{Real}, Matrix{Real}}(eye(length(n)))` : Covariance structure

##### Returns

  * `nodes::Array{Float64}` : An array of quadrature nodes
  * `weights::Array{Float64}` : An array of corresponding quadrature weights

##### Notes

This function has many methods. I try to describe them here.

`n` or `mu` can be a vector or a scalar. If just one is a scalar the other is repeated to match the length of the other. If both are scalars, then the number of repeats is inferred from `sig2`.

`sig2` can be a matrix, vector or scalar. If it is a matrix, it is treated as the covariance matrix. If it is a vector, it is considered the diagonal of a diagonal covariance matrix. If it is a scalar it is repeated along the diagonal as many times as necessary, where the number of repeats is determined by the length of either `n` and/or `mu` (which ever is a vector).

If all 3 are scalars, then 1d nodes are computed. `mu` and `sig2` are treated as the mean and variance of a 1d normal distribution

##### References

Miranda, Mario J, and Paul L Fackler. Applied Computational Economics and Finance, MIT Press, 2002.


In [14]:
?qnwgamma

search: qnwgamma



Computes nodes and weights for beta distribution

##### Arguments

  * `n::Union{Int, Vector{Int}}` : Number of desired nodes along each dimension
  * `a::Union{Real, Vector{Real}}` : Shape parameter of the gamma distribution, along each dimension. Must be positive. Default is 1
  * `b::Union{Real, Vector{Real}}` : Scale parameter of the gamma distribution, along each dimension. Must be positive. Default is 1

##### Returns

  * `nodes::Array{Float64}` : An array of quadrature nodes
  * `weights::Array{Float64}` : An array of corresponding quadrature weights

##### Notes

If any of the parameters to this function are scalars while others are vectors of length `n`, the the scalar parameter is repeated `n` times.

##### References

Miranda, Mario J, and Paul L Fackler. Applied Computational Economics and Finance, MIT Press, 2002.


#### Root finding and optimization

QuantEcon.jl provides a handful of routines for solving or optimizing univariate functions.

The solvers are: `brent` `brenth`, `bisect` and `ridder` 

In [15]:
f2(x) = x^2 - 1
for solver in [brent, brenth, bisect, ridder]
    @show solver, solver(f2, 0.5, 2.0)
end

(solver, solver(f2, 0.5, 2.0)) = (QuantEcon.brent, 1.0000000000001794)
(solver, solver(f2, 0.5, 2.0)) = (QuantEcon.brenth, 0.9999999999999903)
(solver, solver(f2, 0.5, 2.0)) = (QuantEcon.bisect, 1.0000000000002274)
(solver, solver(f2, 0.5, 2.0)) = (QuantEcon.ridder, 1.0000000000005003)


The optimization routine is `golden_method`

In [16]:
# golden_method finds a max, let's find the min of x^2 - 1 between -1 and 1 (which is 0.0)
xstar, f2star = golden_method(x -> -f2(x), -1, 1)

(7.071019315919172e-9, 1.0)